# <center>Sophia Hart, MS, MEng, MS<center>
## <center>sophia.t.hart@gmail.com<center>
# <center>TensorFlow Final Project<center>
## <center>Prof. Alexander I. Iliev, PhD<center>
## <center>UC Berkeley, Spring 2022<center>

## Goal and Data:
Wildfire is a davastating problem in California due to climate change. Last year, in 2021, there were 185 wildfires in California. Wildfires are usually detected by satellite and camera towel images. Human looking at these images causes fatiuge and error.

In this project, I downloaded fire and nonfire images from the internet, and built a convolution neural network by transfer learning with the pretrained Xception model to classify if an image shows there is or there is no wildfire. 


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
import glob
import random

print("TensorFlow version: ", tf.__version__)

%load_ext tensorboard

ModuleNotFoundError: No module named 'tensorflow'

## Divide all jpeg files into training and testing filenames

In [ ]:

# Gather all the files from the fire and nonfire directories
# Some file names are jpg and some are jpeg
paths = glob.glob("./*fire/*.jp*g")

# Shuffle them because top is fire and bottom is nonfire images
random.shuffle(paths)

print("Total number of images ", len(paths))


In [ ]:
# filenames for train and test sets

train_filenames = []
test_filenames = []
i = 0

for filename in paths:  
    
     # Send 25% (1 out of 4) of the images to testing set, the rest to training set
    if i % 4 == 0:
        test_filenames.append(filename)
    else: train_filenames.append(filename) 
    
    i += 1
      
print("Original number of files for training set: ", len(train_filenames), train_filenames)
print("Original number of files for testing set: ", len(test_filenames), test_filenames)

## Create Training and Testing Datasets

In [ ]:
# Define function to create training and testing dataset
# data_filenames is train_filenames or test_filenames.

def create_dataset(data_filenames):
    
    # Create empty list for images and target (ie label: 1 is fire, 0 is nonfire)
    # The lists will be converted into tensor at the end
    image_dataset = []
    target = []    
    a = 0    # index for plotting subplot
    axes = []
    rows = 10
    cols = 10
    fig = plt.figure(figsize = (50, 50))
    ax = plt.gca()
    
    for filename in data_filenames:
        # Read each image file
        image_file = tf.io.read_file(filename)
            
        # try/catch if a file is not jpeg 
        try:
            image = tf.image.decode_jpeg(image_file)
        except:
            print("Not a jpeg file: ", image_file)
        
        # Split filename by "/". The category of fire or nonfire is the second element on the filename
        category = filename.split("/")[1]
        if category == "fire":
            label = 1
        else: label = 0
      
        # Resize image, Xception model expect size 224x224 images
        # Can't do resize_with_crop_or_pad, not compatible with Xception preprocess_input()
        #resize_image = tf.image.resize_with_crop_or_pad(image, 224, 224)
        resize_image = tf.image.resize(image, [224, 224])
   
        # Xception model needs run data through preprocess_input function
        final_image = keras.applications.xception.preprocess_input(resize_image)
        
        # Write image and label to datset
        image_dataset.append(np.array(final_image))
        target.append(label)
    
        # Display image with title (fire or nonfire category that's associated)
        axes.append(fig.add_subplot(rows,cols, a+1))
        plt.imshow(resize_image/255)
        plt.title(category, fontsize=50)
        plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[]) # get rid of ticks on x-axis and y-axis
        a += 1
      
       
        # Below is different data augmentation to increase dataset counts
        
        # flip left-right
        image_flipLR = tf.image.flip_left_right(resize_image)
        final_image = keras.applications.xception.preprocess_input(image_flipLR)
        # Write image and label to datset, still the same label
        image_dataset.append(np.array(final_image))
        target.append(label)

        # flipt up-down
        image_flipUD = tf.image.flip_up_down(resize_image)
        final_image = keras.applications.xception.preprocess_input(image_flipUD)
        image_dataset.append(np.array(final_image))
        target.append(label)
    
        # flipt left-right then up-down
        image_flipLRUD = tf.image.flip_up_down(image_flipLR)
        final_image = keras.applications.xception.preprocess_input(image_flipLRUD)
        image_dataset.append(np.array(final_image))
        target.append(label)
        
    return image_dataset, target
             
# Call the create_dataset() function to create train and test datasets
train_image_dataset, train_target = create_dataset(train_filenames)

test_image_dataset, test_target = create_dataset(test_filenames)

# Convert train_data from a list to a tensor
train_image_dataset = tf.convert_to_tensor(train_image_dataset, dtype=tf.float32)
print('Training image dataset shape: ', train_image_dataset.shape)

# Do the same for train_target
train_target = tf.convert_to_tensor(train_target, dtype=tf.uint8)

# Convert test_data from a list to tensor
test_image_dataset = tf.convert_to_tensor(test_image_dataset, dtype=tf.float32)
print('Testing image dataset shape: ', test_image_dataset.shape)

# Do the same for test_target
test_target = tf.convert_to_tensor(test_target, dtype=tf.uint8)


# print(test_image_dataset)
# --> output of this looks right

## Save datasets into files

In [3]:
# Save datasets into a file for later usage

np.save('train_image_dataset.npy', train_image_dataset)
np.save('train_target.npy', train_target)
np.save('test_image_dataset.npy', test_image_dataset)
np.save('test_target.npy', test_target)


NameError: name 'np' is not defined

## Load datasets from files

In [ ]:
# Load datasets from saved files

train_image_dataset = np.load("train_image_dataset.npy")
print("Shape of train data: ", train_image_dataset.shape)
print(type(train_image_dataset))

test_image_dataset = np.load("test_image_dataset.npy")
print("Shape of test data: ", test_image_dataset.shape)

train_target = np.load("train_target.npy")
print("Shape of train target", train_target.shape)

test_target = np.load("test_target.npy")
print("Shape of test target", test_target.shape)

# Convert train_data from a list to a tensor
train_image_dataset = tf.convert_to_tensor(train_image_dataset, dtype=tf.float32)
print('Training image dataset shape: ', train_image_dataset.shape)
print(type(train_image_dataset))
print(train_image_dataset.dtype)

# Do the same for train_target
train_target = tf.convert_to_tensor(train_target, dtype=tf.uint8)
print(type(train_target))

# Convert test_data from a list to tensor
test_image_dataset = tf.convert_to_tensor(test_image_dataset, dtype=tf.float32)
print('Testing image dataset shape: ', test_image_dataset.shape)
print(type(test_image_dataset))
print(test_image_dataset.dtype)

# Do the same for test_target
test_target = tf.convert_to_tensor(test_target, dtype=tf.uint8)
print(type(test_target))

## Pretrained Model for Transfer Learning
### Using Xception model from ImageNet

In [ ]:
# Use Xception as the base model

base_model = keras.applications.xception.Xception(weights = 'imagenet', include_top=False)

base_model.summary()

In [ ]:
# Define model

n_classes = 2    # fire and nonfire

# Output from the base_model
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)

# Only two classes for softmax
output = keras.layers.Dense(n_classes, activation="softmax")(avg)

# Combine base_model and softmax for our model
model = keras.Model(inputs = base_model.input, outputs = output)

# Trainable variables
print("Trainable variables ", len(model.trainable_variables))

model.summary()

In [ ]:
# Setup for tensorboard

# Create log directory
import os
logdir = './logs/'

os.makedirs(logdir, exist_ok=True)

In [ ]:
# Remove previous event files
!rm -rf ./logs/train/event*

In [ ]:
tensorboard_callback = keras.callbacks.TensorBoard(logdir)

In [ ]:
# Freeze the weights of the base model, ie not trainable

for layer in base_model.layers:
    layer.trainable = False

# Compile model. 
# Have to use sparse_categorical_crossentropy for compile to work
optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.01)

model.compile(loss = "sparse_categorical_crossentropy", 
              optimizer = optimizer, 
              metrics = ['accuracy'])

# Train model
history = model.fit(
    train_image_dataset, 
    train_target, 
    epochs = 5,
    callbacks = [tensorboard_callback])

## TensorBoard

In [ ]:
!ls -l ./logs/train

In [ ]:
# Show accuracy and loss with each epoch in TensorBoard

%tensorboard --logdir ./logs/train

## Evaluate the Model Prediction

In [ ]:
# Evaluate the model

# Predictions is a matrix. Row is the instance, column is the output
predictions = model.predict(x=test_image_dataset, batch_size=60)
print(predictions)

# Use argmax to pull out the column with highest probability, 
# which is the most probable class
predictions = tf.argmax(predictions, axis=-1)
print("Target: ", test_target)
print("Predict ", predictions)

In [ ]:
# Compare predictions and test_target tensors to calculate accuracy

m = tf.keras.metrics.Accuracy()
m.update_state(predictions, test_target)
accuracy = m.result().numpy()

print("Prediction accurcay is: ", accuracy)

# Display confusion matrix
print("Confusion Matrix below: ")

tf.math.confusion_matrix(test_target, predictions)

## Model Fine-tuning: 
### 1. Unfreeze top layers of Xception base model

In [ ]:
# Improve model by unfreezing top layers of the base model, 
# set weights to trainable 

# Total layers of base model
print("Number of layers in base model ", len(base_model.layers))


In [ ]:
# Unfreeze all layers then freeze top layers
base_model.trainable = True

# Fine-tune from this layer ownward
fine_tune_at = 120

# Freeze all layers before the fine-tune-at layer
for layer in base_model.layers[: fine_tune_at]:
    layer.trainable = False
    
# Now trainable variables in base model
print("Trainable Variables in base_model ", len(base_model.trainable_variables))

In [ ]:
# Building model

n_classes = 2

# Output from the base_model
avg2 = keras.layers.GlobalAveragePooling2D()(base_model.output)

# Only two classes for softmax
output2 = keras.layers.Dense(n_classes, activation="softmax")(avg2)

# Combine base_model and softmax for our model
model2 = keras.Model(inputs = base_model.input, outputs = output2)

# Trainable variables
print("Trainable variables ", len(model2.trainable_variables))

# Compile model. Use a slower learning rate and smaller dacay for learning rate
optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.01)
model2.compile(loss = "sparse_categorical_crossentropy",
             optimizer = optimizer,
             metrics = ["accuracy"])

# Tensorboard
tensorboard_callback = keras.callbacks.TensorBoard(logdir)

# Train model
# Run less epoch (4) because it's computational expansive when unfreeze top layers with more images
history = model2.fit(train_image_dataset, 
                    train_target, 
                    epochs=3, 
                    callbacks=[tensorboard_callback])


In [ ]:
# Evaluate the model

# Predictions is a matrix. Row is the instance, column is the output
predictions = model2.predict(x=test_image_dataset, batch_size=60)
predictions = tf.argmax(predictions, axis=-1)

print("Target: ", test_target)
print("Predict ", predictions)

m = tf.keras.metrics.Accuracy()
m.update_state(predictions, test_target)
accuracy = m.result().numpy()
 
print("Prediction accurcay is: ", accuracy)

tf.math.confusion_matrix(test_target, predictions)


## Model Fine-tuning: 
### 2. Add a dropout layer to Xception base model

In [ ]:
# Define model
base_model = keras.applications.xception.Xception(weights = 'imagenet', include_top=False)

n_classes = 2    # fire and nonfire

# Connect base model
avg3 = keras.layers.GlobalAveragePooling2D()(base_model.output)

# Connect dropout3 
dropout3 = keras.layers.Dropout(0.5)(avg3) # Also tried dropout rate = 0.25 and 0.8 (similar results)

# Only two classes for softmax
output3 = keras.layers.Dense(n_classes, activation="softmax")(dropout3)

# Combine base_model and softmax for our model
model3 = keras.Model(inputs = base_model.input, outputs = output3)

# Trainable variables
print("Trainable variables ", len(model3.trainable_variables))

model3.summary()

In [ ]:
# Freeze all layers again

for layer in base_model.layers:
    layer.trainable = False


# Compile the model and start training

# Compile model. Use large learning rate at the begining
optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.01)
model3.compile(loss = "sparse_categorical_crossentropy", 
              optimizer = optimizer, 
              metrics = ['accuracy'])

# Train model
history = model3.fit(
    train_image_dataset, 
    train_target, 
    epochs = 5,
    callbacks = [tensorboard_callback])

In [ ]:
# Evaluate the model

# Predictions is a matrix. Row is the instance, column is the output
predictions = model3.predict(x=test_image_dataset, batch_size=60)
predictions = tf.argmax(predictions, axis=-1)

print("Target: ", test_target)
print("Predict ", predictions)

m = tf.keras.metrics.Accuracy()
m.update_state(predictions, test_target)
accuracy = m.result().numpy()
 
print("Prediction accurcay is: ", accuracy)

tf.math.confusion_matrix(test_target, predictions)


## Model Fine-tuning:
### 3. Change opimizer

In [ ]:
# Define model

n_classes = 2    # fire and nonfire

# Output from the base_model
avg4 = keras.layers.GlobalAveragePooling2D()(base_model.output)

# Only two classes for softmax
output4 = keras.layers.Dense(n_classes, activation="softmax")(avg4)

# Combine base_model and softmax for our model
model4 = keras.Model(inputs = base_model.input, outputs = output4)

# Trainable variables
print("Trainable variables ", len(model4.trainable_variables))


In [ ]:
# Freeze the weights of the base model, ie not trainable

for layer in base_model.layers:
    layer.trainable = False

# Compile model. Use large learning rate at the begining
optimizer = keras.optimizers.Adam()
optimizer.learning_rate = 0.01

model4.compile(loss = "sparse_categorical_crossentropy", 
              optimizer = optimizer, 
              metrics = ['accuracy'])

# Train model
history = model4.fit(
    train_image_dataset, 
    train_target, 
    epochs = 5,
    callbacks = [tensorboard_callback])

In [ ]:
# Evaluate the model

# Predictions is a matrix. Row is the instance, column is the output
predictions = model4.predict(x=test_image_dataset, batch_size=28)
predictions = tf.argmax(predictions, axis=-1)

print("Target: ", test_target)
print("Predict ", predictions)

m = tf.keras.metrics.Accuracy()
m.update_state(predictions, test_target)
accuracy = m.result().numpy()
 
print("Prediction accurcay is: ", accuracy)

tf.math.confusion_matrix(test_target, predictions)


## Results: 
### Accuracy on unseen test image data for multiple runs


| Total images | Run 1 accuracy | Run 2 | Run 3| Run 4 | Run 5 | Run 6 |
| --- | --- | --- | --- | --- | --- | --- |
| 108 | 80% | 83% |  NA |  NA | NA  | NA  |
| 240 | 93% | 96% | 100% | 96% | NA | NA  |
| 320 | 97% | 91% | 97% | 95% | NA | NA | 
| 400 | 96% | 95% | 87% | 98% | 97% | 99% - 100% | 

In [ ]:
# Plotting the results of accuracy on test images for multiple runs.
import matplotlib.pyplot as plt

#x = [108, 240, 320, 400]
x = [240, 320, 400]
run1 = [93, 97.5, 96]
run2 = [96.6, 91, 95]
#x3 = [240, 320, 400]
run3 = [100, 97.5, 87]
run4 = [96.6, 95, 98]
x5 = [400]
run5 = [97]
run6 = [100]

plt.scatter(x, run1)
plt.scatter(x, run2)
plt.scatter(x, run3)
plt.scatter(x, run4)
plt.scatter(x5, run5)
plt.scatter(x5, run6)

plt.xlabel("Total Images", fontsize=20)
plt.ylabel("Accuracy", fontsize=20)
plt.title("Accuracy on Test Images", fontsize=30)

# Average accuracy across 14 runs for 240, 320 and 400 images (exclude data with only 108 images)
accuracy = [ 93, 97.5, 96, 96.6, 91, 95, 100, 97.5, 87, 96.6, 95, 98, 97, 100]
avg = round(sum(accuracy) / len(accuracy), 2)
print("Average accuracy across 12 runs is ", avg)



# Summary


**Creating Dataset** 

At first, I attempted to create TFRecord and extract data from it. However, I encountered dimensionality mismatch when extracted each example for training. So I had this alternative method. 

There are two folders with fire and nonfire images, each has 50 images to balance the two classes. After shuffling, 25% of the images were sent for testing and 75% for training. I wrote a create_dataset() function to convert the images into nested list which is then converted into tensor for dataset. The labels/targets corresponding to the images are saved into another file with the same order as the images. Each image is resized to (224, 224). The channels are preserved because color is important in detecting the wildfire. Data augmentation is applied by flipping images left-right and up-down to quadruple into 400 images.

**Model**

Deep convolution neural network is used for the model.  I built my model base on the pretrained Xception base model (proposed in 2016 by Francois Chollet, author of Keras), that has 132 layers, and then I added 2 or 3 layers on top of it. The weights are extracted from imagenet training and they are set as non-trainable. Xception model has depthwise separable convolution layers, which assume spatial patterns and cross-channel patterns can be modeled separately [1]. This saves computation time.   

**Results**

I graduatlly increase the data size, 108, 240, 320 and 400 total images after augmentation. The table and plot above show the best accuracy on test images among 4 different models-- initial model and three fine-tuned models. The accuracies were above 90%, exception one outlier at 87% for 400 images but 5 others times fall between 95% - 100%.  The outlier could be because there are one or two images that are very ambigeous that got into the test set, this becomes 4 or 8 images after augmentation. The accuracy for 108 images are low, but they were just quick test that my code works when I was developing models.

I tried three different ways to fine tune the model: 1. unfreezed the top layers, 2. added a dropout layer at the top layer, 3. changed the optimizer. Out of at least 12 times I ran the program, different strategies can do better or worse than the initial model.

The more data I have, the wider distribution the data can be, so it is not black-and-white (fire or nonfire). The model is robus enough to give excellent predictions. Thanks to the Xception model that I was able to transfer the learning. Computer vision with pretrained model is amazing in accuracy and simplicity!

## References:

1. Geron, Aurelien, Hands-on Machine Learning with SciKit-Learn, Keras & TensorFlow, 2nd Ed, O'Reilly